# AY2022/23 IS3107 Mini Project Local Db Setup 
- Name: Andre Heng Zhe Hau
- email: e0725806@u.nus.edu
- Matric No.: A0233806H

### Setup

### Connect to Cassandra

In [139]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()


### Keyspace setup 

In [140]:
# Drop Keyspace localdb
session.execute('DROP KEYSPACE IF EXISTS localdb;')

# Create Keyspace localdb
session.execute(
    "CREATE KEYSPACE IF NOT EXISTS localdb WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor': '1'}")
session.set_keyspace('localdb')

# Create and Register UDTs
session.execute(
    "CREATE TYPE author (author_name text, orcid text, pid text)")
create_position = '''
        CREATE TYPE position (
                number text,
                volume text,
                pages text
            );
        '''
session.execute(create_position)
cluster.register_user_type('localdb', 'author', dict)
cluster.register_user_type('localdb', 'position', dict)


### Drop Tables 

In [141]:
# Drop
# 0
drop_table_pubs_by_pid = '''
            DROP TABLE IF EXISTS localdb.pubs_by_pid; 
            '''
session.execute(drop_table_pubs_by_pid)
# 1
drop_table_pubs_by_pid_cat_year_index = '''
            DROP TABLE IF EXISTS localdb.pubs_by_pid_cat_year_index; 
            '''
session.execute(drop_table_pubs_by_pid_cat_year_index)
# 2
drop_table_pubs_by_pid_year_coauthors_pid = '''
            DROP TABLE IF EXISTS localdb.pubs_by_pid_year_coauthors_pid;
            '''
session.execute(drop_table_pubs_by_pid_year_coauthors_pid)
# 3
drop_table_pubs_by_name_year_coauthors_pid = '''
            DROP TABLE IF EXISTS localdb.pubs_by_name_year_coauthors_pid;
            '''
session.execute(drop_table_pubs_by_name_year_coauthors_pid)
# 4
drop_table_pubs_by_pid_index_cat_year = '''
            DROP TABLE IF EXISTS localdb.pubs_by_pid_index_cat_year;
            '''
session.execute(drop_table_pubs_by_pid_index_cat_year)
# 5
drop_table_pubs_by_pid_index_year = '''
            DROP TABLE IF EXISTS localdb.pubs_by_pid_index_year;
            '''
session.execute(drop_table_pubs_by_pid_index_year)
# 6
drop_table_pubs_name_coauthors_pid = '''
            DROP TABLE IF EXISTS localdb.pubs_name_coauthors_pid;
            '''
session.execute(drop_table_pubs_name_coauthors_pid)


### Create Tables

In [142]:
# pubs_by_pid
create_table_pubs_by_pid = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_pid (
                pid text,
                paper_key text,
                authorEntryIndex int,
                category text,
                year int,
                author_name text,
                authors map<int, frozen<author>>,
                title text,
                mdate date,
                PRIMARY KEY (pid, paper_key) 
            ) WITH CLUSTERING ORDER BY (paper_key ASC) AND 
            comment = 'pubs_by_pid';'''
session.execute(create_table_pubs_by_pid)

# pubs_by_pid_cat_year_index
create_table_pubs_by_pid_cat_year_index = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_pid_cat_year_index (
                pid text,
                paper_key text,
                authorEntryIndex int,
                category text,
                year int,
                author_name text,
                authors map<int, frozen<author>>,
                title text,
                mdate date,
                PRIMARY KEY (pid, category, year, authorEntryIndex, paper_key) 
            ) WITH CLUSTERING ORDER BY (category ASC, year DESC, authorEntryIndex ASC, paper_key ASC) AND 
            comment = 'pubs_by_pid_cat_year_index';'''
session.execute(create_table_pubs_by_pid_cat_year_index)

# pubs_by_pid_year_coauthors_pid
create_table_pubs_by_pid_year_coauthors_pid = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_pid_year_coauthors_pid (
                pid text,
                author_name text,
                coauthor_pid text,
                paper_key text,
                year int,
                title text,
                mdate date,
                PRIMARY KEY ((pid, year), coauthor_pid, paper_key)
            ) WITH CLUSTERING ORDER BY (coauthor_pid ASC, paper_key ASC) AND
            comment = 'pubs_by_pid_year_coauthors_pid';'''
session.execute(create_table_pubs_by_pid_year_coauthors_pid)

# pubs_by_name_year_coauthors_pid
create_table_pubs_by_name_year_coauthors_pid = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_name_year_coauthors_pid (
                pid text,
                author_name text,
                coauthor_pid text,
                paper_key text,
                year int,
                title text,
                mdate date,
                PRIMARY KEY ((author_name, year), coauthor_pid, pid, paper_key)
            ) WITH CLUSTERING ORDER BY (coauthor_pid ASC, pid ASC, paper_key ASC) AND
            comment = 'pubs_by_name_year_coauthors_pid';'''
session.execute(create_table_pubs_by_name_year_coauthors_pid)

# pubs_by_pid_index_cat_year
create_table_pubs_by_pid_index_cat_year = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_pid_index_cat_year (
                pid text,
                paper_key text,
                authorEntryIndex int,
                category text,
                year int,
                author_name text,
                authors map<int, frozen<author>>,
                title text,
                mdate date,
                PRIMARY KEY (pid, authorEntryIndex, category, year, paper_key) 
            ) WITH CLUSTERING ORDER BY (authorEntryIndex ASC, category ASC, year DESC, paper_key ASC) AND 
            comment = 'pubs_by_pid_index_cat_year';'''
session.execute(create_table_pubs_by_pid_index_cat_year)

# pubs_by_pid_index_year
create_table_pubs_by_pid_index_year = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_pid_index_year (
                pid text,
                paper_key text,
                authorEntryIndex int,
                category text,
                year int,
                author_name text,
                authors map<int, frozen<author>>,
                title text,
                mdate date,
                PRIMARY KEY (pid, authorEntryIndex, year, paper_key) 
            ) WITH CLUSTERING ORDER BY (authorEntryIndex ASC, year DESC, paper_key ASC) AND 
            comment = 'pubs_by_pid_index_year';'''
session.execute(create_table_pubs_by_pid_index_year)

# pubs_by_name_coauthors_pid
create_table_pubs_by_name_coauthors_pid = '''CREATE TABLE IF NOT EXISTS localdb.pubs_by_name_coauthors_pid (
                pid text,
                author_name text,
                coauthor_pid text,
                paper_key text,
                year int,
                title text,
                mdate date,
                PRIMARY KEY (author_name, coauthor_pid, pid, paper_key)
            ) WITH CLUSTERING ORDER BY (coauthor_pid ASC, pid ASC, paper_key ASC) AND
            comment = 'pubs_by_name_coauthors_pid';'''
session.execute(create_table_pubs_by_name_coauthors_pid)


### Prepare Insert Statements

In [143]:
#index
insert_data_pubs_by_pid = '''
    INSERT INTO localdb.pubs_by_pid (
        pid,
        paper_key,
        authorEntryIndex,
        category,
        year,
        author_name,
        authors,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt1 = session.prepare(insert_data_pubs_by_pid)

insert_data_pubs_by_pid_index_year = '''
    INSERT INTO localdb.pubs_by_pid_index_year (
        pid,
        paper_key,
        authorEntryIndex,
        category,
        year,
        author_name,
        authors,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt2 = session.prepare(insert_data_pubs_by_pid_index_year)

insert_data_pubs_by_pid_index_cat_year = '''
    INSERT INTO localdb.pubs_by_pid_index_cat_year (
        pid,
        paper_key,
        authorEntryIndex,
        category,
        year,
        author_name,
        authors,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt3 = session.prepare(insert_data_pubs_by_pid_index_cat_year)

insert_data_pubs_by_pid_cat_year_index = '''
    INSERT INTO localdb.pubs_by_pid_cat_year_index (
        pid,
        paper_key,
        authorEntryIndex,
        category,
        year,
        author_name,
        authors,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt4 = session.prepare(insert_data_pubs_by_pid_cat_year_index)

#coauthor_pid
insert_data_pubs_by_name_coauthors_pid = '''
    INSERT INTO localdb.pubs_by_name_coauthors_pid (
        pid,
        author_name,
        coauthor_pid,
        paper_key,
        year,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt5 = session.prepare(insert_data_pubs_by_name_coauthors_pid)

insert_data_pubs_by_pid_year_coauthors_pid = '''
    INSERT INTO localdb.pubs_by_pid_year_coauthors_pid (
        pid,
        author_name,
        coauthor_pid,
        paper_key,
        year,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt6 = session.prepare(insert_data_pubs_by_pid_year_coauthors_pid)

insert_data_pubs_by_name_year_coauthors_pid = '''
    INSERT INTO localdb.pubs_by_name_year_coauthors_pid (
        pid,
        author_name,
        coauthor_pid,
        paper_key,
        year,
        title,
        mdate
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    '''
insert_stmt7 = session.prepare(insert_data_pubs_by_name_year_coauthors_pid)

### Insert Initial Data

In [100]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET 

cs_researchers = pd.read_csv("/Users/andre/Plan3 Design & Build Dropbox/Andre Heng/Mac/Documents/NUS Y2S1/IS3107/IS3107 Mini Project/cs_researchers.csv")

def check_if_pub_exists(paper_key):

    create_idx_q1 = '''CREATE INDEX IF NOT EXISTS ON pubs_by_pid (paper_key);'''

    probe_query = f'''
        SELECT title FROM pubs_by_pid WHERE paper_key = '{paper_key}';
    '''
    session.execute(create_idx_q1)
    rows = session.execute(probe_query)
    check = []
    for (title) in rows:    
        check.append({'title':title})
    check = pd.DataFrame(check)
    if check.empty:
        return True
    else:
        return False

def extract_from_xml(file_to_process):
    pub_list = []
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    type_of_pub_arr = ("article", "inproceedings", "proceedings", "book", "incollection", "phdthesis", "masterthesis", "www", "person", "data") 
    type_of_author_arr = ("author", "editor")

    for dblpperson in root.iter('dblpperson'):
        pid = dblpperson.attrib['pid']
        author_name = dblpperson.attrib['name']

    for item in root.findall('r'):

        for child in item:

            pub_attribs = {}
            pub_attribs["pid"] = pid
            pub_attribs["author_name"] = author_name

            for i in type_of_pub_arr:
                type = item.find(i)
                if type is not None:
                    paper_key = type.attrib['key']
                    pub_attribs["paper_key"] = paper_key
                    pub_attribs["mdate"] = type.attrib['mdate']
                    break

            coauthors_pid = []
            authors = {}
            count = 1
            for i in type_of_author_arr:
                for auth in child.findall(i):
                    name = auth.text
                    if auth.get('orcid'):
                        orcid = auth.attrib['orcid']
                    else:
                        orcid = None
                    diff_pid = auth.attrib['pid']
                    if diff_pid == pub_attribs["pid"]:
                        pub_attribs["authorEntryIndex"] = count
                    else:
                        coauthors_pid.append(diff_pid)
                    
                    authors[count] = (name, orcid, diff_pid)
                    count += 1
                    
                pub_attribs["authors"] = authors
                pub_attribs["coauthors_pid"] = coauthors_pid
                # if pub_attribs['authorEntryIndex'] is None:
                #     pub_attribs['authorEntryIndex'] = "HAHAHAHAHAHBABABABABABABA~!!!"

            pub_attribs["category"] = pub_attribs["paper_key"].split('/')[0].rstrip('s')

            year = child.find('year')
            if year is not None:
                year = int(year.text)
                pub_attribs["year"] = year

            title = child.find("title")
            if title is not None:
                title = title.text
                pub_attribs["title"] = title

            ees = set()
            for one_ee in child.findall("ee"):
                ees.add(one_ee.text)            
            pub_attribs["ee"] = ees
            
            #do check here before appending
            if check_if_pub_exists(paper_key) == True:
                pub_list.append(pub_attribs)
                # print(pub_list)
            # print(pub_list)
    # print(pub_list)
    return pub_list

def load_pub_ls(pub_ls):
    for pub in pub_ls:
        session.execute(insert_stmt1,
                        [pub['pid'], pub['paper_key'], pub['authorEntryIndex'], pub['category'], pub['year'], pub['author_name'], pub['authors'], pub['title'], pub['mdate']])

        session.execute(insert_stmt2,
                        [pub['pid'], pub['paper_key'], pub['authorEntryIndex'], pub['category'], pub['year'], pub['author_name'], pub['authors'], pub['title'], pub['mdate']])

        session.execute(insert_stmt3,
                        [pub['pid'], pub['paper_key'], pub['authorEntryIndex'], pub['category'], pub['year'], pub['author_name'], pub['authors'], pub['title'], pub['mdate']])

        session.execute(insert_stmt4,
                        [pub['pid'], pub['paper_key'], pub['authorEntryIndex'], pub['category'], pub['year'], pub['author_name'], pub['authors'], pub['title'], pub['mdate']])

        for coauthor_pid in pub['coauthors_pid']:
                session.execute(insert_stmt5,
                                [pub['pid'], pub['author_name'], coauthor_pid, pub['paper_key'], pub['year'], pub['title'], pub['mdate']])

                session.execute(insert_stmt6,
                                [pub['pid'], pub['author_name'], coauthor_pid, pub['paper_key'], pub['year'], pub['title'], pub['mdate']])

                session.execute(insert_stmt7,
                                [pub['pid'], pub['author_name'], coauthor_pid, pub['paper_key'], pub['year'], pub['title'], pub['mdate']])
    return pub_ls

api = "https://dblp.org/pid/"
for index, row in cs_researchers.iterrows():
    pid = row['PID']
    name = row['Name']
    current_api = api + pid
    current_api += ".xml"

    response = requests.get(current_api)
    if response.status_code == 200:
        with open('xmlfile.xml', 'wb+') as f:
            f.write(response.content)
            f.close()

        pub_ls = extract_from_xml('xmlfile.xml')
        load_pub_ls(pub_ls)

### Connect to AstraDB

In [144]:
from cassandra.auth import PlainTextAuthProvider

#append files accordingly
cloud_config= {
         'secure_connect_bundle': '/Users/andre/Plan3 Design & Build Dropbox/Andre Heng/Mac/Documents/NUS Y2S1/IS3107/IS3107 Mini Project/secure-connect-is3107astradb.zip'
}
auth_provider = PlainTextAuthProvider('XZGMaIyaQCJBSOZOXBxQHGrX', '7KhXw2ywLrDsJDNryIP1mTohNE5lDgxccpqjKjl17EPDIAhY,-WbqTKv8,k6IZ.hrKxr-g5xZ2eL39QBywTAJZCyO3j5euY2mWS85dpQxh9qYdLZ7QwsaUsh,hrTbATT')


### Astra DB Setup

In [145]:
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('astradb')

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

# Create and Register UDTs
session.execute(
    "CREATE TYPE author (author_name text, orcid text, pid text)")
create_position = '''
    CREATE TYPE position (
            number text,
            volume text,
            pages text
        );
    '''
session.execute(create_position)
cluster.register_user_type('astradb', 'author', dict)
cluster.register_user_type('astradb', 'position', dict)

4.0.0.6816


### Drop AstraDB Tables

In [146]:
# Drop
# 0
drop_table_volume_update = '''
            DROP TABLE IF EXISTS volume_update; 
            '''
session.execute(drop_table_volume_update)
# 1
drop_table_author_pub_update = '''
            DROP TABLE IF EXISTS author_pub_update; 
            '''
session.execute(drop_table_author_pub_update)
# 2
drop_table_query_outcomes = '''
            DROP TABLE IF EXISTS query_outcomes;
            '''
session.execute(drop_table_query_outcomes)

In [147]:
#create tables
create_tab_1 = '''
CREATE TABLE IF NOT EXISTS volume_update (
    timestamp text,
    num_new_pubs int,
    num_unique_pubs text,
    PRIMARY KEY (timestamp)
) WITH comment='astradb volume update log';
'''
session.execute(create_tab_1)

create_tab_2 = '''
CREATE TABLE IF NOT EXISTS author_pub_update (
    timestamp text,
    paper_key text,
    title text,
    authors map<int, frozen<author>>, 
    ee set<text>,
    PRIMARY KEY (timestamp, paper_key)
) WITH comment='astradb author pub update log';
'''
session.execute(create_tab_2)

create_tab_3 = '''
CREATE TABLE IF NOT EXISTS query_outcomes (
    query_num text,
    query_outcome text,
    PRIMARY KEY (query_num)
) WITH comment='query_outcomes';
'''
session.execute(create_tab_3)

### Query and Insert Initial Data 2.2

In [149]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('localdb')

import pandas as pd

def query_answers():
    query_results = []
    # Q1
    rows = session.execute(
        "SELECT authorEntryIndex, COUNT(*) as total, category FROM pubs_by_pid_index_cat_year WHERE pid = '40/2499' AND authorEntryIndex = 3 AND category = 'conf' AND year >= 2012 AND year <= 2022;")
    query_entry_1 = {}
    query_entry_1["query_num"] = 'Q1'
    q1 = []
    for (authorEntryIndex, total, category) in rows:
        q1.append({'authorEntryIndex': authorEntryIndex,
                  'total': total, 'category': category})
    q1 = pd.DataFrame(q1)
    q1 = q1.reset_index(drop=True).iloc[0].to_string()
    query_entry_1["query_outcome"] = q1
    query_results.append(query_entry_1)

    # Q2
    rows = session.execute(
        "SELECT authorEntryIndex, COUNT(*) as total, pid FROM pubs_by_pid_index_year WHERE pid = 'o/BengChinOoi' AND authorEntryIndex = 2 AND year >= 2017 AND year <= 2022;")
    query_entry_1 = {}
    query_entry_1["query_num"] = 'Q2'
    q2 = []
    for (authorEntryIndex, total, pid) in rows:
        q2.append({'authorEntryIndex': authorEntryIndex,
                  'total': total, 'pid': pid})
    q2 = pd.DataFrame(q2)
    q2 = q2.reset_index(drop=True).iloc[0].to_string()
    query_entry_1["query_outcome"] = q2
    query_results.append(query_entry_1)

    #Q3
    rows = session.execute("SELECT COUNT(*) as total, author_name FROM pubs_by_name_coauthors_pid WHERE author_name='Lihua Xie' GROUP BY coauthor_pid;")
    query_entry_1 = {}
    query_entry_1["query_num"] = 'Q3'
    q3 = []
    input_total = 0
    for (total, author_name) in rows:
        input_total += total
    q3.append({'total':input_total, 'author_name':author_name})
    q3 = pd.DataFrame(q3)
    q3 = q3.reset_index(drop=True).iloc[0].to_string()
    query_entry_1["query_outcome"] = q3
    query_results.append(query_entry_1)

    #Q4
    rows = session.execute("SELECT coauthor_pid, count(*) as total, year FROM pubs_by_name_year_coauthors_pid WHERE author_name='Beng Chin Ooi' AND year = 2020 GROUP BY coauthor_pid;")
    query_entry_1 = {}
    query_entry_1["query_num"] = 'Q4'
    q4 = []
    for (coauthor_pid, total, year) in rows:    
        q4.append({'coauthor_pid':coauthor_pid, 'year':year, 'total':total})
    q4 = pd.DataFrame(q4)
    q4 = q4.sort_values(by=['total'],ascending=False).reset_index(drop=True).iloc[0].to_string()
    query_entry_1["query_outcome"] = q4
    query_results.append(query_entry_1)

    return query_results

query_results = query_answers()
print(query_results)

[{'query_num': 'Q1', 'query_outcome': 'authorEntryIndex       3\ntotal                 53\ncategory            conf'}, {'query_num': 'Q2', 'query_outcome': 'authorEntryIndex                2\ntotal                           4\npid                 o/BengChinOoi'}, {'query_num': 'Q3', 'query_outcome': 'total               1905\nauthor_name    Lihua Xie'}, {'query_num': 'Q4', 'query_outcome': 'coauthor_pid    67/6383-1\nyear                 2020\ntotal                   7'}]


### Insert Query Data to Astra DB

In [110]:
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('astradb')

#Insert Query Results
insert_query_data = '''
    INSERT INTO astradb.query_outcomes (
        query_num,
        query_outcome
    ) VALUES (?, ?)
'''
insert_stmt_query = session.prepare(insert_query_data)

for query in query_results:
    session.execute(insert_stmt_query,
                    [query['query_num'], query['query_outcome']])